Get data

In [1]:
!pip install arch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Test

In [1]:
import pandas as pd
import numpy as np
from arch import arch_model

# Reload with semicolon separator and convert decimal commas
df = pd.read_csv(
    r"C:\Users\cberr\OneDrive\Bureau\Rendements CAC40.csv",
    sep=';',
    decimal=',',  
    parse_dates=["Date"]
)

# Cleaning column names
df.columns = df.columns.str.strip()

# Explicit conversion of numeric columns
df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
df["EPU"] = pd.to_numeric(df["EPU"], errors="coerce")

# Set date as index
df = df.set_index("Date")
df = df.sort_index()

# Calculate logarithmic returns
df["returns"] = np.log(df["Close"] / df["Close"].shift(1))
df = df.dropna()

# Display
print(df[["Close", "returns", "EPU"]].head())






              Close   returns         EPU
Date                                     
2010-01-05  4012.91 -0.000264  147.080379
2010-01-06  4017.67  0.001185  147.080379
2010-01-07  4024.80  0.001773  147.080379
2010-01-08  4045.14  0.005041  147.080379
2010-01-11  4043.09 -0.000507  147.080379


GARCH CAC 40

In [2]:
import pandas as pd
import numpy as np
from arch import arch_model

# Reload with semicolon separator and convert decimal commas
df = pd.read_csv(
    r"C:\Users\cberr\OneDrive\Bureau\Rendements CAC40.csv",
    sep=';',
    decimal=',',  # ← allows interpreting 4013,97 as a float
    parse_dates=["Date"]
)

# Cleaning column names
df.columns = df.columns.str.strip()

# Explicit conversion of numeric columns
df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
df["EPU"] = pd.to_numeric(df["EPU"], errors="coerce")

# Set date as index
df = df.set_index("Date")
df = df.sort_index()

# Calculate logarithmic returns
df["returns"] = np.log(df["Close"] / df["Close"].shift(1))

# Rescaling returns (important for GARCH convergence)
df["returns_scaled"] = df["returns"] * 100

df = df.dropna()

# Lag the EPU (EPU_t-1)
df["EPU_lag1"] = df["EPU"].shift(1)

# Remove missing values only for the columns used
df = df[["returns_scaled", "EPU_lag1"]].dropna()

# GARCH-X model
model = arch_model(
    df["returns_scaled"],
    vol="GARCH",
    p=1,
    q=1,
    mean="ARX",        
    lags=0,  
    x=df[["EPU_lag1"]]
)

# Model estimation
results = model.fit(disp="off")

# Summary
print(results.summary())
print("\n--- Estimated Coefficients ---")
print(results.params)

# Display the coefficient of interest
if "EPU_lag1" in results.params.index:
    print(f"\nCoefficient EPU_t-1: {results.params['EPU_lag1']:.4f}")
else:
    print(" EPU_t-1 not included in the mean.")



                          AR-X - GARCH Model Results                          
Dep. Variable:         returns_scaled   R-squared:                      -0.001
Mean Model:                      AR-X   Adj. R-squared:                 -0.002
Vol Model:                      GARCH   Log-Likelihood:               -5871.93
Distribution:                  Normal   AIC:                           11753.9
Method:            Maximum Likelihood   BIC:                           11785.2
                                        No. Observations:                 3920
Date:                Sun, Jun 29 2025   Df Residuals:                     3918
Time:                        14:43:47   Df Model:                            2
                                  Mean Model                                 
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
Const      5.5560e-03  4.754e-02      0.117      0.907 

GARCH DAX

In [3]:
import pandas as pd
import numpy as np
from arch import arch_model

df = pd.read_csv(
    r"C:\Users\cberr\OneDrive\Bureau\Rendements DAX.csv",
    sep=';',
    decimal=',', 
    parse_dates=["Date"]
)

df.columns = df.columns.str.strip()

df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
df["EPU"] = pd.to_numeric(df["EPU"], errors="coerce")

df = df.set_index("Date")
df = df.sort_index()

df["returns"] = np.log(df["Close"] / df["Close"].shift(1))

df["returns_scaled"] = df["returns"] * 100

df = df.dropna()

df["EPU_lag1"] = df["EPU"].shift(1)

df = df[["returns_scaled", "EPU_lag1"]].dropna()

model = arch_model(
    df["returns_scaled"],
    vol="GARCH",
    p=1,
    q=1,
    mean="ARX",        
    lags=0,  
    x=df[["EPU_lag1"]]
)

results = model.fit(disp="off")

print(results.summary())
print("\n--- Estimated Coefficients ---")
print(results.params)

if "EPU_lag1" in results.params.index:
    print(f"\nEPU_t-1 Coefficient: {results.params['EPU_lag1']:.4f}")
else:
    print("EPU_t-1 not included in the mean.")



                          AR-X - GARCH Model Results                          
Dep. Variable:         returns_scaled   R-squared:                      -0.001
Mean Model:                      AR-X   Adj. R-squared:                 -0.001
Vol Model:                      GARCH   Log-Likelihood:               -5826.16
Distribution:                  Normal   AIC:                           11662.3
Method:            Maximum Likelihood   BIC:                           11693.6
                                        No. Observations:                 3887
Date:                Sun, Jun 29 2025   Df Residuals:                     3885
Time:                        14:44:13   Df Model:                            2
                                  Mean Model                                 
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
Const          0.0646  2.631e-02      2.454  1.414e-02 

GARCH UKX

In [4]:
import pandas as pd
import numpy as np
from arch import arch_model

df = pd.read_csv(
    r"C:\Users\cberr\OneDrive\Bureau\Rendements UKX.csv",
    sep=';',
    decimal=',',  
    parse_dates=["Date"]
)

df.columns = df.columns.str.strip()

df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
df["EPU"] = pd.to_numeric(df["EPU"], errors="coerce")

df = df.set_index("Date")
df = df.sort_index()

df["returns"] = np.log(df["Close"] / df["Close"].shift(1))

df["returns_scaled"] = df["returns"] * 100

df = df.dropna()

df["EPU_lag1"] = df["EPU"].shift(1)

df = df[["returns_scaled", "EPU_lag1"]].dropna()

model = arch_model(
    df["returns_scaled"],
    vol="GARCH",
    p=1,
    q=1,
    mean="ARX",        
    lags=0,  
    x=df[["EPU_lag1"]]
)

results = model.fit(disp="off")

print(results.summary())
print("\n--- Estimated Coefficients ---")
print(results.params)

if "EPU_lag1" in results.params.index:
    print(f"\nEPU_t-1 Coefficient: {results.params['EPU_lag1']:.4f}")
else:
    print("EPU_t-1 not included in the mean.")


                          AR-X - GARCH Model Results                          
Dep. Variable:         returns_scaled   R-squared:                      -0.001
Mean Model:                      AR-X   Adj. R-squared:                 -0.001
Vol Model:                      GARCH   Log-Likelihood:               -5247.83
Distribution:                  Normal   AIC:                           10505.7
Method:            Maximum Likelihood   BIC:                           10537.0
                                        No. Observations:                 3866
Date:                Sun, Jun 29 2025   Df Residuals:                     3864
Time:                        14:44:30   Df Model:                            2
                                  Mean Model                                  
                  coef    std err          t      P>|t|       95.0% Conf. Int.
------------------------------------------------------------------------------
Const           0.0596  3.617e-02      1.647  9.956e

GARCH IBEX

In [5]:
import pandas as pd
import numpy as np
from arch import arch_model

df = pd.read_csv(
    r"C:\Users\cberr\OneDrive\Bureau\Rendements IBEX.csv",
    sep=';',
    decimal=',',  
    parse_dates=["Date"]
)

df.columns = df.columns.str.strip()

df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
df["EPU"] = pd.to_numeric(df["EPU"], errors="coerce")

df = df.set_index("Date")
df = df.sort_index()

df["returns"] = np.log(df["Close"] / df["Close"].shift(1))

df["returns_scaled"] = df["returns"] * 100

df = df.dropna()

df["EPU_lag1"] = df["EPU"].shift(1)

df = df[["returns_scaled", "EPU_lag1"]].dropna()

model = arch_model(
    df["returns_scaled"],
    vol="GARCH",
    p=1,
    q=1,
    mean="ARX",        
    lags=0,  
    x=df[["EPU_lag1"]]
)

results = model.fit(disp="off")

print(results.summary())
print("\n--- Estimated Coefficients ---")
print(results.params)

if "EPU_lag1" in results.params.index:
    print(f"\nEPU_t-1 Coefficient: {results.params['EPU_lag1']:.4f}")
else:
    print("EPU_t-1 not included in the mean.")


                          AR-X - GARCH Model Results                          
Dep. Variable:         returns_scaled   R-squared:                      -0.001
Mean Model:                      AR-X   Adj. R-squared:                 -0.001
Vol Model:                      GARCH   Log-Likelihood:               -6218.05
Distribution:                  Normal   AIC:                           12446.1
Method:            Maximum Likelihood   BIC:                           12477.5
                                        No. Observations:                 3916
Date:                Sun, Jun 29 2025   Df Residuals:                     3914
Time:                        14:44:44   Df Model:                            2
                                  Mean Model                                 
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
Const          0.0185  9.124e-02      0.203      0.839 

GARCH FTSEMIB

In [6]:
import pandas as pd
import numpy as np
from arch import arch_model

df = pd.read_csv(
    r"C:\Users\cberr\OneDrive\Bureau\Rendements FTSEMIB.csv",
    sep=';',
    decimal=',',  
    parse_dates=["Date"]
)

df.columns = df.columns.str.strip()

df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
df["EPU"] = pd.to_numeric(df["EPU"], errors="coerce")

df = df.set_index("Date")
df = df.sort_index()

df["returns"] = np.log(df["Close"] / df["Close"].shift(1))

df["returns_scaled"] = df["returns"] * 100

df = df.dropna()

df["EPU_lag1"] = df["EPU"].shift(1)

df = df[["returns_scaled", "EPU_lag1"]].dropna()

model = arch_model(
    df["returns_scaled"],
    vol="GARCH",
    p=1,
    q=1,
    mean="ARX",         
    lags=0,  
    x=df[["EPU_lag1"]]
)

results = model.fit(disp="off")

print(results.summary())
print("\n--- Estimated Coefficients ---")
print(results.params)

if "EPU_lag1" in results.params.index:
    print(f"\nEPU_t-1 Coefficient: {results.params['EPU_lag1']:.4f}")
else:
    print("EPU_t-1 not included in the mean.")


                          AR-X - GARCH Model Results                          
Dep. Variable:         returns_scaled   R-squared:                      -0.001
Mean Model:                      AR-X   Adj. R-squared:                 -0.001
Vol Model:                      GARCH   Log-Likelihood:               -6587.53
Distribution:                  Normal   AIC:                           13185.1
Method:            Maximum Likelihood   BIC:                           13216.4
                                        No. Observations:                 3889
Date:                Sun, Jun 29 2025   Df Residuals:                     3887
Time:                        14:45:01   Df Model:                            2
                                  Mean Model                                  
                  coef    std err          t      P>|t|       95.0% Conf. Int.
------------------------------------------------------------------------------
Const           0.1052  5.891e-02      1.786  7.407e

GARCH EuroStoxx

In [7]:
import pandas as pd
import numpy as np
from arch import arch_model

df = pd.read_csv(
    r"C:\Users\cberr\OneDrive\Bureau\Rendements EuroStoxx.csv",
    sep=';',
    decimal=',',  
    parse_dates=["Date"]
)

df.columns = df.columns.str.strip()

df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
df["EPU"] = pd.to_numeric(df["EPU"], errors="coerce")

df = df.set_index("Date")
df = df.sort_index()

df["returns"] = np.log(df["Close"] / df["Close"].shift(1))

df["returns_scaled"] = df["returns"] * 100

df = df.dropna()

df["EPU_lag1"] = df["EPU"].shift(1)

df = df[["returns_scaled", "EPU_lag1"]].dropna()

model = arch_model(
    df["returns_scaled"],
    vol="GARCH",
    p=1,
    q=1,
    mean="ARX",         
    lags=0,  
    x=df[["EPU_lag1"]]
)

results = model.fit(disp="off")

print(results.summary())
print("\n--- Estimated Coefficients ---")
print(results.params)

if "EPU_lag1" in results.params.index:
    print(f"\nEPU_t-1 Coefficient: {results.params['EPU_lag1']:.4f}")
else:
    print("EPU_t-1 not included in the mean.")


                          AR-X - GARCH Model Results                          
Dep. Variable:         returns_scaled   R-squared:                      -0.001
Mean Model:                      AR-X   Adj. R-squared:                 -0.001
Vol Model:                      GARCH   Log-Likelihood:               -5968.34
Distribution:                  Normal   AIC:                           11946.7
Method:            Maximum Likelihood   BIC:                           11978.1
                                        No. Observations:                 3930
Date:                Sun, Jun 29 2025   Df Residuals:                     3928
Time:                        14:45:14   Df Model:                            2
                                  Mean Model                                 
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
Const          0.0372  5.323e-02      0.699      0.484 